## STEP 1. 형태소 분석기를 이용하여 품사에 따라 해당 단어를 추출하기

- 명사만 추출 `nouns.txt`
- 명사, 동사 추출 `nouns_verbs.txt`
- 명사, 동사, 형용사 추출 `nouns_verbs_adjectives.txt`

In [1]:
# 시간이 오래 소요되므로 파일로 저장
# word2vec 생성을 위한 토큰화
import os
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
input_file_path = os.getenv('HOME') + '/aiffel/weat/synopsis.txt'
output_file_path = './tokenized/nouns.txt'

'''
with open(input_file_path, 'r', encoding='utf-8') as rf:
    with open(output_file_path, 'w', encoding='utf-8') as wf:
        lines = rf.readlines()
        for line in tqdm(lines):
            words = okt.pos(line, stem=True, norm=True)
            res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]
            wf.write(' '.join(res) + '\n')
'''

'\nwith open(input_file_path, \'r\', encoding=\'utf-8\') as rf:\n    with open(output_file_path, \'w\', encoding=\'utf-8\') as wf:\n        lines = rf.readlines()\n        for line in tqdm(lines):\n            words = okt.pos(line, stem=True, norm=True)\n            res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]\n            wf.write(\' \'.join(res) + \'\n\')\n'

In [2]:
# 저장해놓은 토큰 파일 불러오기
tokenized=[]
with open(output_file_path, 'r', encoding='utf-8') as rf:
    for line in rf:
        # 각 줄을 읽어 공백을 기준으로 단어를 나누고 리스트로 변환
        words = line.strip().split()
        tokenized.append(words)

In [3]:
# 개수 세기
cnt=0
for words in tokenized:
    cnt+=len(words)
print(cnt)

1342179


- Noun 개수: 1342179
- Noun & Verb 개수: 1723183
- Noun & Verb & Adjective 개수: 1850315

## STEP 2. 추출된 결과로 embedding model 만들기

In [4]:
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])

[('작품', 0.8968769907951355),
 ('다큐멘터리', 0.8536402583122253),
 ('영화로', 0.8201987743377686),
 ('드라마', 0.8191244006156921),
 ('형식', 0.7980725169181824),
 ('주제', 0.7934190630912781),
 ('코미디', 0.7801860570907593),
 ('감동', 0.7790406942367554),
 ('소재', 0.7768155336380005),
 ('인터뷰', 0.7672218084335327)]

## STEP 3. target, attribute 단어 셋 만들기

In [5]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from konlpy.tag import Okt

# 토큰화 파일 저장 함수
def save_tokens(input_file_name, output_file_name):
    okt = Okt()
    result = []
    with open(os.getenv('HOME')+'/aiffel/weat/'+input_file_name, 'r') as fread: 
        with open('./tokenized/'+output_file_name, 'w', encoding='utf-8') as fwrite:
            print(input_file_name, '파일을 읽고 있습니다.')
            lines = fread.readlines()
            for line in tqdm(lines):
                words = okt.pos(line, stem=True, norm=True)
                res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]
                fwrite.write(' '.join(res) + '\n')
# 토큰 파일 불러오는 함수              
def read_tokens(file_name):
    with open('./tokenized/' + file_name, 'r', encoding='utf-8') as fread:
        # 파일 전체를 읽어서 줄바꿈 문자를 제거하고 하나의 문자열로 반환
        tokenized = fread.read().replace('\n', ' ')
    
    return tokenized

명사만 추출할 때에는 `okt.nouns()`을 활용해보았지만 소요 시간 차이 크지 않았음

In [6]:
# 불용어 처리 함수
with open('./stopwords.txt', 'r', encoding='utf-8') as fread:
    stopwords = [line.strip() for line in fread.readlines()]
    print(stopwords)
    
def remove_stopwords(tokenized, stopwords):
    words = tokenized.split()
    filtered_words = [word for word in words if word not in stopwords]
    result = ' '.join(filtered_words)
    
    return result

['하다', '되다', '그녀', '자신', '위해', '않다', '이다', '되어다', '싶다', '점점', '때문', '오다', '가다', '과연', '이제', '통해']


stopwords 구성 기준: 모든 데이터셋에서 자주 등장하고, 주제와 관련 없는 단어 선정

### target 단어 추출

In [7]:
# target 데이터 불러오기
art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'
# 품사에 따라 바꾸기
output_art_txt = 'synopsis_art_nouns.txt'
output_gen_txt = 'synopsis_gen_nouns.txt'

'''
save_tokens(art_txt, output_art_txt)
save_tokens(gen_txt, output_gen_txt)
'''

art=read_tokens(output_art_txt)
gen=read_tokens(output_gen_txt)

In [8]:
# 불용어 제거
art=remove_stopwords(art, stopwords)
gen=remove_stopwords(gen, stopwords)

In [10]:
vec_type = 'count'

# TF-IDF
if vec_type=='tfidf':
    vectorizer = TfidfVectorizer()
# TF
elif vec_type=='count':
    vectorizer = CountVectorizer()
    
X = vectorizer.fit_transform([art,gen])

In [11]:
m1 = X[0].tocoo()   # art를 TF-IDF로 표현한 sparse matrix를 가져옵니다. 
m2 = X[1].tocoo()   # gen을 TF-IDF로 표현한 sparse matrix를 가져옵니다. 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 

w1_, w2_ = [], []
print('예술영화를 대표하는 단어들:')
for i in range(100):
    word = vectorizer.get_feature_names()[w1[i][0]]
    print(word, end=', ')
    w1_.append(word)

print('\n')
    
print('일반영화를 대표하는 단어들:')
for i in range(100):
    word = vectorizer.get_feature_names()[w2[i][0]]
    print(word, end=', ')
    w2_.append(word)

예술영화를 대표하는 단어들:
시작, 사랑, 사람, 영화, 친구, 남자, 가족, 

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


이야기, 마을, 사건, 마음, 세상, 아버지, 아이, 엄마, 모든, 여자, 대한, 서로, 시간, 다시, 아들, 소녀, 아내, 다른, 영화제, 사이, 세계, 사실, 하나, 남편, 감독, 여행, 인생, 발견, 모두, 순간, 우리, 가장, 마지막, 아빠, 생활, 모습, 기억, 죽음, 비밀, 학교, 음악, 한편, 소년, 생각, 도시, 명의, 결혼, 사고, 전쟁, 위기, 최고, 이자, 과거, 일상, 경찰, 간다, 상황, 미국, 운명, 결심, 관계, 현실, 지금, 단편, 여인, 하루, 이름, 이후, 준비, 인간, 만난, 감정, 처음, 국제, 누구, 살인, 충격, 동안, 존재, 그린, 어머니, 연인, 계속, 동생, 작품, 한국, 청년, 가지, 상처, 할머니, 목숨, 이상, 희망, 

일반영화를 대표하는 단어들:
영화제, 사람, 시작, 국제, 영화, 친구, 사랑, 남자, 이야기, 대한, 서울, 여자, 사건, 남편, 아이, 가족, 아버지, 다른, 마을, 시간, 엄마, 아들, 모든, 단편, 마음, 사실, 다시, 세계, 모습, 작품, 생각, 서로, 세상, 발견, 감독, 아내, 관계, 소녀, 사이, 하나, 우리, 애니메이션, 여성, 죽음, 인간, 생활, 한편, 결혼, 상황, 모두, 기억, 명의, 소년, 여행, 가장, 간다, 순간, 도시, 비밀, 학교, 과거, 가지, 이자, 경찰, 마지막, 미국, 동안, 전쟁, 주인공, 대해, 존재, 현실, 연출, 사고, 살인, 일상, 어머니, 계속, 사회, 인생, 다큐멘터리, 부문, 섹스, 최고, 바로, 의도, 동생, 하루, 위기, 계획, 정체, 한국, 이후, 조직, 회사, 보고, 부산, 아빠, 부부, 일본, 

> Q. 왜 두 문서 사이에 중복값이 많을까?  
> A. 둘 다 영화를 다루고 있으므로 공통된 어휘가 많이 사용됨. 또한 현재 문서가 2개만 존재하므로 단어 간의 IDF 값 차이가 적어져 TF 값이 TF-IDF 값에 영향을 끼치는 정도가 커짐

In [12]:
# 중복 제거
n = 15

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

In [13]:
print('예술영화를 대표하는 단어들:')
for word in target_art:
    print(word, end=', ')

print('\n')
    
print('일반영화를 대표하는 단어들:')
for word in target_gen:
    print(word, end=', ')

예술영화를 대표하는 단어들:
음악, 운명, 결심, 지금, 여인, 이름, 준비, 만난, 감정, 처음, 누구, 충격, 그린, 연인, 청년, 

일반영화를 대표하는 단어들:
서울, 애니메이션, 여성, 주인공, 대해, 연출, 사회, 다큐멘터리, 부문, 섹스, 바로, 의도, 계획, 정체, 조직, 

### attribute 단어 추출 

In [14]:
#파일 불러와서 TF-IDF 생성
genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
             'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
             'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
             'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
             'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
         '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지']
# 품사에 따라 바꾸기
output_genre_txt = [txt.strip('.txt')+'_nouns.txt' for txt in genre_txt]

genre = []
for file_name, output_file_name in zip(genre_txt, output_genre_txt):
    #save_tokens(file_name, output_file_name)
    genre.append(read_tokens(output_file_name))

In [15]:
# 불용어 제거
genre_ = []
for gen in genre:
    genre_.append(remove_stopwords(gen, stopwords))

In [16]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(genre_)

In [17]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)

attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names()[w[i][j][0]])
            print(vectorizer.get_feature_names()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

SF: 지구, 시작, 사람, 인류, 인간, 미래, 우주, 로봇, 세계, 모든, 박사, 우주선, 외계, 존재, 세상, 
가족: 엄마, 아빠, 가족, 영화제, 친구, 아주르, 아버지, 시작, 아들, 마을, 국제, 낙타, 할머니, 씨제이, 동구, 
공연: 오페라, 사랑, 토스카, 실황, 올레, 카바, 공연, 오텔로, 리골레토, 백작, 프레, 베르디, 카르피, 비바, 왕자, 
공포(호러): 시작, 사람, 친구, 사건, 공포, 발견, 죽음, 마을, 가족, 악령, 남자, 좀비, 영화, 사실, 소녀, 
기타: 영화제, 국제, 서울, 단편, 영화, 사람, 이야기, 남자, 시작, 사랑, 뉴미디어, 페스티벌, 여자, 대한, 독립, 
다큐멘터리: 영화제, 영화, 다큐, 국제, 다큐멘터리, 사람, 이야기, 대한, 감독, 서울, 우리, 시작, 세계, 여성, 가족, 
드라마: 영화제, 사람, 사랑, 영화, 시작, 국제, 남자, 친구, 이야기, 엄마, 여자, 아버지, 가족, 단편, 서울, 
멜로로맨스: 사랑, 시작, 남편, 남자, 여자, 사람, 친구, 섹스, 마음, 결혼, 서로, 아내, 관계, 부부, 엄마, 
뮤지컬: 뮤지컬, 사랑, 에스메랄다, 음악, 충무로, 모차르트, 영화, 토스카, 니웨, 카바, 영화제, 바흐, 페뷔스, 프롤, 모도, 
미스터리: 사건, 시작, 사람, 발견, 사고, 진실, 죽음, 기억, 살인, 친구, 아내, 남자, 아이, 민혁, 사실, 
범죄: 사건, 경찰, 시작, 범죄, 조직, 살인, 사람, 마약, 형사, 남자, 모든, 살해, 수사, 발견, 한길수, 
사극: 조선, 시작, 신기전, 사랑, 아가멤논, 황제, 루안, 최고, 운명, 사람, 하선, 전쟁, 윤서, 트로이, 세자, 
서부극(웨스턴): 서부, 보안관, 벌린, 카우보이, 그레이프바인, 헨리, 마을, 개릿, 아이, 시작, 무법자, 프린트, 마적, 태구, 현상금, 
성인물(에로): 남편, 마사지, 섹스, 관계, 영화, 정사, 남자, 시작, 여자, 유부녀, 마음, 사랑, 에피소드, 그린, 아내, 
스릴러: 사건, 

## STEP 4. WEAT score 계산과 시각화

In [ ]:
from numpy import dot
from numpy.linalg import norm
# WEAT score function
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))
def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B #, c_a, c_b
def weat_score(X, Y, A, B):    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

In [ ]:
# embedding model과 단어 셋으로 WEAT score 구해보기
matrix = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]
X = np.array([model.wv[word] for word in target_art])
Y = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in attributes[i]])
        B = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)
        
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

In [ ]:
# 시각화
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def plot_heatmap(matrix, genre_name, title, save=True):
    np.random.seed(0)
    # 한글 지원 폰트
    sns.set(font='NanumGothic')

    # 마이너스 부호 
    plt.rcParams['axes.unicode_minus'] = False

    fig, ax = plt.subplots(figsize=(15,13))
    sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True, cmap='RdYlGn_r', ax=ax)

    ax.set_title(title, fontsize=20, pad=20)
    if save:
        plt.savefig('heatmap_'+title+'.png', bbox_inches='tight')

    plt.show()
    
plot_heatmap(matrix, genre_name, 'simple_tfidf', True)